<div style="background-color:white; color:black; padding:10px; width:100%;">
  <table style="width:100%; border:none;">
    <tr style="border:none;">
      <td style="text-align:left; border:none; vertical-align:middle;">
        <a href="https://dswa.vercel.app/" target="_blank">
          <img src="images/logo.svg" alt="Logo" style="height:120px;">
        </a>
      </td>
      <td style="text-align:right; border:none; vertical-align:middle;">
        <h2 style="margin:0; font-size:80px;">
          <strong href="https://dswa.vercel.app/" target="_blank" style="text-decoration:none; color:black; padding-right:40px;">
            DSWA
          </strong>
        </h2>
      </td>
    </tr>
  </table>
</div>

# Segmentação de Clientes com K-Means

**Tópico:** Aprendizado Não Supervisionado - Análise de Cluster

**Dataset:** [Mall Customer Segmentation Data](https://www.kaggle.com/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python/data)

### **Objetivo da Competição**

Bem-vindo(a)!

Neste notebook, vamos explorar o mundo do **Aprendizado Não Supervisionado**. Diferente do aprendizado supervisionado (onde temos um "gabarito" ou uma variável alvo, como "vendeu" ou "não vendeu"), aqui o objetivo é encontrar padrões e estruturas escondidas nos dados por conta própria.

Usaremos uma das técnicas mais famosas para isso: o **K-Means Clustering**. Nossa missão será agrupar clientes de um shopping com base em seus perfis de compra, criando "segmentos" ou "personas".

**A sua tarefa** será analisar esses segmentos e extrair insights de negócio a partir deles. Vamos começar!

### **Passo 1: Preparação do Ambiente**

Primeiro, vamos importar as bibliotecas que nos ajudarão nesta jornada.

-   **pandas:** para carregar e manipular nossos dados.
-   **numpy:** para operações numéricas.
-   **matplotlib & seaborn:** para criar visualizações incríveis.
-   **scikit-learn:** para usar o algoritmo K-Means.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### **Passo 2: Carregando e Explorando os Dados (EDA - Exploratory Data Analysis)**

Vamos usar o dataset "Mall Customer Segmentation" do Kaggle. Ele contém informações básicas sobre os clientes de um shopping.

**Colunas do Dataset:**
*   **CustomerID:** ID único para cada cliente.
*   **Gender:** Gênero do cliente.
*   **Age:** Idade do cliente.
*   **Annual Income (k$):** Renda anual do cliente em milhares de dólares.
*   **Spending Score (1-100):** Uma pontuação de 1 a 100 atribuída pelo shopping com base no comportamento de gastos do cliente.

In [ ]:
from io import StringIO

# Carregando o dataset
df = pd.read_csv('data/Mall_Customers.csv')

# Vamos dar uma primeira olhada nos dados
print("As 5 primeiras linhas do nosso dataset:")
display(df.head())

print("\nInformações gerais sobre as colunas:")
df.info()

print("\nVerificando se há valores nulos:")
print(df.isnull().sum())

**Primeiras Impressões:**
*   Temos 200 registros (clientes).
*   As colunas estão com os tipos de dados corretos.
*   Ótima notícia: não temos valores nulos! Isso facilita nosso trabalho.

### **Passo 3: Visualizando os Dados**

A melhor forma de entender os dados é visualizando-os. Vamos criar alguns gráficos para encontrar padrões.

In [ ]:
# Distribuição da Idade, Renda Anual e Score de Gastos
plt.figure(1, figsize=(18, 5))

# Gráfico da Idade
plt.subplot(1, 3, 1)
sns.histplot(df['Age'], bins=20, kde=True, color='blue')
plt.title('Distribuição da Idade')

# Gráfico da Renda Anual
plt.subplot(1, 3, 2)
sns.histplot(df['Annual Income (k$)'], bins=20, kde=True, color='green')
plt.title('Distribuição da Renda Anual (k$)')

# Gráfico do Score de Gastos
plt.subplot(1, 3, 3)
sns.histplot(df['Spending Score (1-100)'], bins=20, kde=True, color='red')
plt.title('Distribuição do Score de Gastos (1-100)')

plt.show()

**Análise dos Gráficos de Distribuição:**
*   **Idade:** A maior concentração de clientes está entre 20 e 40 anos.
*   **Renda Anual:** A maioria dos clientes tem uma renda entre 40k e 80k dólares.
*   **Score de Gastos:** A pontuação de gastos é mais concentrada no meio, entre 40 e 60.

Agora, vamos focar no que mais nos interessa: a relação entre **renda** e **score de gastos**. Um gráfico de dispersão (scatter plot) é perfeito para isso.

In [ ]:
# Gráfico de Dispersão: Renda Anual vs. Score de Gastos
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', data=df, s=100, alpha=0.7)
plt.title('Renda Anual vs. Score de Gastos')
plt.xlabel('Renda Anual (k$)')
plt.ylabel('Score de Gastos (1-100)')
plt.show()

**Observação Importante:**
Olhando para o gráfico acima, você consegue ver "grupos" ou "aglomerados" de pontos? Parece que existem cerca de 5 grupos distintos de clientes. É exatamente isso que o algoritmo K-Means vai tentar formalizar para nós!

### **Passo 4: Preparando os Dados e Encontrando o Número Ideal de Clusters**

O K-Means precisa saber quantos clusters (o "K" do nome) ele deve criar. Podemos usar uma técnica chamada **Método do Cotovelo (Elbow Method)** para nos ajudar a decidir.

A ideia é rodar o K-Means para vários valores de K (de 1 a 10, por exemplo) e medir a "inércia" de cada um. A inércia é a soma das distâncias quadradas de cada ponto ao centro do seu cluster. Queremos uma inércia baixa.

O "cotovelo" no gráfico é o ponto onde a inércia para de diminuir drasticamente. Esse é um bom indicativo do número ideal de clusters.

In [ ]:
# Vamos focar nas duas variáveis que visualizamos: Renda e Score.
X = df[['Annual Income (k$)', 'Spending Score (1-100)']].values

# Lista para armazenar a inércia (WCSS - Within-Cluster Sum of Squares)
wcss = []

for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# Plotando o gráfico do Método do Cotovelo
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('WCSS (Inércia)')
plt.xticks(range(1, 11))
plt.show()

**Análise do Gráfico do Cotovelo:**
O "cotovelo" da curva é claramente no ponto **K=5**. A partir daí, a redução na inércia se torna muito menor. Portanto, 5 é o número ideal de clusters para nossos dados. Nossa intuição visual estava certa!

### **Passo 5: Aplicando o K-Means e Visualizando os Segmentos**

Agora que sabemos o número mágico (K=5), vamos aplicar o algoritmo e ver os resultados.

In [ ]:
# Criando e treinando o modelo K-Means com 5 clusters
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42, n_init=10)
y_kmeans = kmeans.fit_predict(X)

# Adicionando a coluna de 'Cluster' ao nosso DataFrame original
df['Cluster'] = y_kmeans

# Visualizando os clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', hue='Cluster', data=df, palette='viridis', s=100, alpha=0.9)

# Plotando os centroides (o "coração" de cada cluster)
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1], s=300, c='red', marker='X', label='Centroides')

plt.title('Segmentação de Clientes com K-Means')
plt.xlabel('Renda Anual (k$)')
plt.ylabel('Score de Gastos (1-100)')
plt.legend()
plt.show()

**Sucesso!** O algoritmo K-Means separou nossos clientes em 5 grupos distintos e visualmente coerentes. Agora vem a parte mais importante: entender o que cada grupo significa.

---

## **SUA VEZ**

Agora que os clusters foram criados, sua tarefa é se tornar um(a) analista de dados e transformar esses grupos em insights de negócio valiosos.

### **Tarefa 1: Análise e Interpretação dos Clusters**

Para cada um dos 5 clusters, você deve descrever o perfil dos clientes que pertencem a ele. Uma ótima maneira de começar é calculando a média das características para cada grupo.

**Dica:** Use o método `.groupby()` do pandas.

**Responda às seguintes perguntas (no seu relatório ou em uma célula de texto abaixo):**

1.  **Crie um "apelido" ou "persona" para cada cluster.** Por exemplo: "Econômicos", "Gastadores", "Público Alvo", "Pão-duro", etc. Justifique sua escolha com base nos dados.
2.  **Descreva em detalhes o perfil de cada cluster.**
    *   **Cluster 0:** Qual é a principal característica deste grupo? (Ex: Renda alta, Score baixo)
    *   **Cluster 1:** ...
    *   **Cluster 2:** ...
    *   **Cluster 3:** ...
    *   **Cluster 4:** ...
3.  **Qual cluster você considera o mais valioso para o shopping? E o menos valioso?** Por quê?

In [ ]:
# Treinando o modelo definitivo com 5 clusters
kmeans_final = KMeans(n_clusters=5, init='k-means++', random_state=42)
y_kmeans = kmeans_final.fit_predict(X)

# Adicionando os rótulos ao DataFrame original para análise
df['Cluster'] = y_kmeans

# Visualizando os Clusters
plt.figure(figsize=(10, 6))

# Loop para plotar cada cluster com uma cor diferente
colors = ['red', 'blue', 'green', 'cyan', 'magenta']
labels = ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']

for i in range(5):
    plt.scatter(X[y_kmeans == i, 0], X[y_kmeans == i, 1], 
                s=100, c=colors[i], label=labels[i])

# Plotando os Centróides
plt.scatter(kmeans_final.cluster_centers_[:, 0], kmeans_final.cluster_centers_[:, 1], 
            s=300, c='yellow', label='Centróides', edgecolors='black')

plt.title('Segmentação de Clientes (Renda x Score)')
plt.xlabel('Renda Anual (k$)')
plt.ylabel('Score de Gastos (1-100)')
plt.legend()
plt.show()

# Agrupamos pelo 'Cluster' e calculamos a média das variáveis
analise_personas = df.groupby('Cluster')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].mean()
contagem_pessoas = df['Cluster'].value_counts()

print("\n--- Perfil Médio de Cada Cluster ---")
display(analise_personas)
print("\n--- Quantidade de Clientes por Cluster ---")
display(contagem_pessoas)

### Respostas da Tarefa 1: Análise Detalhada dos Clusters

1. Personas e Apelidos
Com base na análise dos centróides e das médias, definimos os seguintes perfis:

* **Cluster 0: Convencionais**
    * Possuem renda e score de gastos intermediários. Representam o comportamento padrão do consumidor médio do shopping.
* **Cluster 1: Elite**
    * Combinam alta renda com alto score de gastos. É o grupo que movimenta a receita de luxo.
* **Cluster 2: Herdeiros**
    * Renda baixa, mas score de gastos muito alto. A média de idade (25 anos) indica que podem ser jovens que ainda moram com os pais ou gastam tudo o que ganham em tendências.
* **Cluster 3: Pão-duros**
    * Alta renda, mas score de gastos baixíssimo. Têm dinheiro, mas não veem valor em gastar neste shopping especificamente.
* **Cluster 4: Econômicos**
    * Renda baixa e score de gastos baixo. Compram apenas o necessário ou em momentos de promoção agressiva.

---

2. Perfil Detalhado de Cada Cluster

* **Cluster 0: Convencionais**
    * Renda média (~55k) e Score médio (~50).
    * É o grupo mais numeroso (81 pessoas). São clientes estáveis, mas previsíveis. Não são impulsivos, mas mantêm o fluxo do shopping constante.
* **Cluster 1: Elite**
    * Renda Alta (~86k) e Score Alto (~82).
    * Público maduro (média 32 anos) e financeiramente estabelecido. São o alvo ideal para lojas de grife, serviços exclusivos e programas de fidelidade premium.
* **Cluster 2: Herdeiros**
    * Renda Baixa (~26k) e Score Alto (~79).
    * O grupo mais jovem (média 25 anos). Provavelmente influenciados por moda rápida, tecnologia e lazer (cinema/praça de alimentação).
* **Cluster 3: Pão-duros**
    * Renda Alta (~88k) e Score Baixo (~17).
    * Média de idade de 41 anos. São consumidores racionais. Podememos buscar entender por que eles não gastam: Falta de lojas adequadas ao seu perfil? Preço? Atendimento?
* **Cluster 4: Econômicos**
    * Renda Baixa (~26k) e Score Baixo (~21).
    * Idade média de 45 anos. Devido à restrição orçamentária, evitam gastos supérfluos. Visitam o shopping provavelmente para serviços essenciais ou vitrine, sem conversão de venda imediata.

---

3. Análise de Valor

* **Cluster Mais Valioso: Cluster 1 - Elite**
    * Eles possuem a combinação perfeita de poder aquisitivo + disposição para gastar. O Retorno sobre Investimento em campanhas para este grupo tende a ser o mais alto, pois eles não precisam ser convencidos a abrir a carteira, apenas a direcionar o gasto para as lojas certas.

* **Cluster Menos Valioso: Cluster 4 - Econômicos**
    * Eles têm baixa capacidade de gerar receita devido à baixa renda e baixo engajamento. Esforços de marketing aqui teriam custo alto para um retorno baixo.
    * O Cluster 3 também é um desafio, mas eles têm dinheiro, então são "potencialmente valiosos" se convertidos. O Cluster 4 não tem essa alavanca financeira no momento.

### **Tarefa 2: Desafio - Incluindo a Idade na Análise**

Nosso modelo usou apenas Renda e Score de Gastos. O que acontece se incluirmos a **Idade** na clusterização?

1.  **Repita o processo de clusterização (Passos 4 e 5)**, mas desta vez usando as colunas `Age`, `Annual Income (k$)` e `Spending Score (1-100)`.
2.  Você pode manter K=5 ou usar o Método do Cotovelo novamente para ver se um número diferente de clusters seria melhor com 3 variáveis.
3.  **Analise os novos clusters.** Eles são parecidos com os que você encontrou na Tarefa 1? A idade parece ser um fator importante para separar os grupos?
4.  **Atenção:** Você não poderá mais usar um gráfico de dispersão 2D para visualizar os clusters, pois agora temos 3 dimensões. A análise terá que ser feita principalmente com base nos valores médios de cada cluster (como no `groupby` da Tarefa 1).

In [ ]:
# Preparando os dados 3D
# Selecionamos agora 3 colunas: Age, Annual_Income, Spending_Score
X3 = df[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].values

# 2. Reavaliando o K (Método do Cotovelo para 3D)
wcss_3d = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(X3)
    wcss_3d.append(kmeans.inertia_)

# Plotando o Cotovelo
plt.figure(figsize=(10, 5))
plt.plot(range(1, 11), wcss_3d, marker='o', color='purple')
plt.title('Método do Cotovelo (Considerando a Idade)')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()

# --- DECISÃO DO K ---
# Ao observar o gráfico, a curva suaviza perto de K=6.
# A inclusão da idade cria novas nuances que justificam um grupo extra.

# 3. Treinando o Modelo 3D Final (K=6)
kmeans_3d = KMeans(n_clusters=6, init='k-means++', random_state=42)
y_kmeans_3d = kmeans_3d.fit_predict(X3)

# Adicionando o novo cluster ao DataFrame (coluna diferente para não sobrescrever a anterior)
df['Cluster_3D'] = y_kmeans_3d

# 4. Visualização 3D
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Cores para 6 clusters
colors = ['red', 'blue', 'green', 'cyan', 'magenta', 'orange']

for i in range(6):
    ax.scatter(X3[y_kmeans_3d == i, 1],  # Eixo X: Renda
               X3[y_kmeans_3d == i, 2],  # Eixo Y: Score
               X3[y_kmeans_3d == i, 0],  # Eixo Z: Idade
               s=80, c=colors[i], label=f'Cluster {i}')

ax.set_xlabel('Renda Anual (k$)')
ax.set_ylabel('Score (1-100)')
ax.set_zlabel('Idade')
plt.title('Segmentação 3D: Renda x Score x Idade')
plt.legend()
plt.show()

# 5. Resumo das Personas 3D
print("--- Perfil dos 6 Clusters (Considerando Idade) ---")
display(df.groupby('Cluster_3D')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].mean())
print("\n--- Contagem de Clientes por Cluster 3D ---")
display(df['Cluster_3D'].value_counts())

### Análise dos Resultados: Inclusão da Variável "Age"

**1. Comparação com a Tarefa 1:**
Ao adicionar a Idade, o algoritmo (com K=6) refinou nossa visão, dividindo grupos que antes pareciam únicos.
* Percebemos uma divisão clara baseada na idade entre quem ganha pouco.
    * Cluster 2 - Herdeiros: Média de 25 anos, ganham pouco (~28k) mas gastam muito (Score 77).
    * Cluster 5 - Mais velhos e Econômicos: Média de 45 anos, ganham pouco (~26k) e gastam pouco (Score 21).
    * Conclusão: A idade é o fator determinante aqui. Jovens com baixa renda tendem a gastar (provavelmente dependentes ou sem contas fixas), enquanto os mais velhos com a mesma renda são contidos.

**2. A Idade Influencia a Alta Renda?**
Sim.
* Os grupos de Alto Score (Clusters 1 e 4) são formados por adultos jovens (média de 32-33 anos).
* Já o grupo de Baixo Score (Cluster 3) é formado por pessoas mais maduras (média de 40 anos).
* Isso sugere que o público mais jovem, mesmo com alta renda, é mais propenso ao consumo neste shopping do que o público de meia-idade.

**3. Sobre os Super Ricos:**
O algoritmo também separou os clientes de alta renda e alto score em dois níveis:
* Cluster 4: Renda ~78k (Elite Padrão).
* Cluster 1: Renda ~110k (Super Elite - um grupo pequeno de 10 pessoas com poder de compra extremo).

**Resumo:** A idade provou ser uma variável crucial, especialmente para distinguir o comportamento de consumo nas faixas de renda mais baixa.

### **Tarefa 3: Conclusões e Recomendações de Negócio**

Com base na sua análise da **Tarefa 1**, escreva um parágrafo final com recomendações para a equipe de marketing do shopping.

*   Que tipo de campanha de marketing você criaria para o cluster dos "clientes com alta renda e alto score de gastos"?
*   E para o grupo de "baixa renda e alto score de gastos"?
*   Como o shopping poderia tentar engajar o grupo de "alta renda e baixo score de gastos" para que eles gastem mais?

Com base nos perfis identificados (especialmente considerando a idade), sugerimos as seguintes estratégias para a equipe de marketing:

### Estratégia para "Alta Renda e Alto Score"
* **Perfil:** Adultos jovens (~32 anos) com muito dinheiro e disposição para gastar.
* **Ação Sugerida:** Focar na exclusividade e experiência.
    * Criar um "Clube VIP" com estacionamento gratuito ou valet.
    * Enviar convites para pré-lançamentos de coleções de grifes.
    * Parcerias com restaurantes de alta gastronomia dentro do shopping.

### Estratégia para "Baixa Renda e Alto Score"
* **Perfil:** Jovens (~25 anos), provavelmente estudantes ou em primeiro emprego, que gostam de consumir.
* **Ação Sugerida:** Focar em volume, tendência e descontos.
    * Promoções de "Leve 3, Pague 2" em lojas.
    * Eventos "instagramáveis" e campanhas com influenciadores digitais locais.
    * Cupons de desconto para a praça de alimentação e cinema.

### Estratégia de Conversão: "Alta Renda e Baixo Score"
* **Perfil:** Adultos (~40 anos) com dinheiro, mas que não gastam no shopping. Este é o grupo com maior potencial desperdiçado.
* **Ação Sugerida:** entender e atrair pela conveniência/serviço.
    * Oferecer serviços que facilitem a vida: Centro automotivo, academia premium, supermercado de alta qualidade ou área de bem-estar.
    * Realizar uma pesquisa de satisfação direcionada para entender se o mix de lojas atual atende ao gosto mais maduro desse público.
    * Campanhas focadas em "Qualidade" e não em "Promoção".

---
**Conclusão Final:**
A segmentação revelou que o shopping possui ecossistemas diferentes convivendo no mesmo espaço. Não existe uma campanha única. O segredo para aumentar o faturamento pode ser a personalização: atrair os jovens nas redes sociais, enquanto se oferece luxo e conforto para os VIPs em canais diretos.

---

**Formato de Entrega:**
Entregue este notebook com todas as células executadas e suas respostas para as Tarefas 1, 2 e 3 preenchidas em células de texto (Markdown) e código.

**Boa sorte e boa análise!**